<a href="https://colab.research.google.com/github/karenyxwang/Big_Data_Analytics/blob/master/Spark%20SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 545 Homework 3: Spark SQL
### **Due Date: March 23, 2020**

Welcome to CIS 545 Homework 3! In this homework you will gain a mastery of using Spark SQL. By the end, you'll be a star (not that you aren't already one). Over the next few days you will be using an EMR cluster to use Spark to manipulate the  `linkedin_small_real.json` dataset  as well as the `stocks.csv`.

The goal of the homework will be to create a training dataset for a Random Forest Machine learning model.  Yes, you'll be playing with machine learning shortly!

The training data set will contain the monthly number of employees hired by companies in `linkedin_small_real.json` and their corresponding closing stock prices over a 10 year period (2000-2011). We will try and predict, based on this data, if the company will have a positive or negative growth in stock in the first quarter of the next year. Who's ready to make some money?

## The Necessary Notes and Nags
Before we begin here are some important notes to keep in mind,


1.   **IMPORTANT!** I said it twice, it's really important. In this homework, we will be using AWS resources. You are given a quota ($75) to use for the entirety of the homework. There is a small chance you will use all this money, however it is important that at the end of every session, you **shut down your EMR cluster**.
2.   **Be sure you use Google Colab for this Homework** since we must connect to the EMR cluster and local Jupyter will have issues doing that. Using a Google Colab Notebook with an EMR cluster has two important abnormalities:
    * The first line of any cell in which you will use the spark session must be `%%spark`. Notice that all cells below have this.
    * You will, unfortunately, not be able to stop a cell while it is running. If you wish to do so, you will need to restart your cluster. See the Setup EMR Document for reference.
3.   You are **required** to use Spark SQL queries to handle the data in the assignment. Mastering SQL is more beneficial than being able to use Spark commands (functions) as it will show up in more areas of programming and data science/analytics than just Spark. Use the following [function list](https://spark.apache.org/docs/latest/api/sql/index.html#) to see all the SQL functions avaliable in Spark.
4.   Throughout the homework you will be manipulating Spark dataframes (sdfs). We do not specify any ordering on the final output. You are welcome to order your final tables in whatever way you deem fit. We will conduct our own ordering when we grade.
5. Based on the challenges you've faced in the previous homework, we are including information on the expected schema of your results.  Apache Spark is very fiddly but we hope this will help.
6. There are portions of this homework that are hard. We urge you start early to come to office hours and get help if you get stuck. But don't worry, I can see the future, and you all got this.

With that said, let's dive in.




## Step 0: Set up EMR

Your first task is to create an EMR (Elastic MapReduce) cluster your AWS Educate Accounts. Please see the [attached document](https://cis545hw1data.s3.amazonaws.com/Microsoft+Word+-+cis545_setup_emr+(2).doc.pdf) for detailed instructions.

**There is one very important addition to the instructions:  when you get to page 13 and want to `Create a Cluster`, you need to first click on 'Block public access' (left side of the screen), then *Change* `Block public access` to *Off*.**  Otherwise your cluster will consistently fail to spin up!

Move on to Step 0.1 after you have completed all the steps in the document.

![ACME GIANT RUBBER BAND](https://pbs.twimg.com/media/DRqbJh7UMAE2z4o?format=jpg&name=4096x4096)


### Step 0.1: The Superfluous Setup

Run the following two cells. These will allow your colab notebook to connect to an use your EMR.

In [ ]:
%%capture
!apt update
!apt install gcc python-dev libkrb5-dev
!pip install sparkmagic

In [ ]:
%%capture
%load_ext sparkmagic.magics 

### Step 0.2: The Sharp Spark

Now, connect your notebook to the EMR cluster you created. In the first cell, copy the link to the Master Public DNS specified in the setup document. You will need to add `http://` to the beginning of the address and the port to the end. The final format should be,

`http://<your-DNS-link>:8998`

For example, if my DNS (directly from the AWS EMR console) is `ec2-3-15-237-211.us-east-2.compute.amazonaws.com` my address would be,

`http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com:8998`

Insert this in the `# TODO # below`. For our example, the cell would read,



```
%spark add -s spark_session -l python -u http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com:8998
```

In [ ]:
# TODO: Enter your Master Public DNS with the proper formatting and host

#Change to your instance

%spark add -s my_session1 -l python -u http://ec2-52-91-166-92.compute-1.amazonaws.com:8998

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1585342858839_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [ ]:
# If you ever need to restart, you may need to...
# %spark delete -s my_session
#OR just factory reset runtime under the runtime tab

Enter your 8-digit Penn ID as an integer in the cell 
below. This will be used in the autograder.

In [ ]:
%%spark
from penngrader.grader import *
STUDENT_ID = 15349478

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader = PennGrader(homework_id = 'CIS545_Spring_2020_HW3', student_id = STUDENT_ID)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PennGrader initialized with Student ID: 15349478

Make sure this correct or we will not be able to store your grade

Run the following cell to setup the autograder, make sure to have set your 8 digit Penn ID in the cell above. It will also import all the modules you need for the homework.

_Note_: Since we are using an EMR cluster we will only have access to some of modules that exist for Python, meaning things like `pandas`, `numpy`, etc. may not all be available. We have written the entire homework such that the solution does not require any of these. Don't try to import them. It won't work.

## Step 1: Data Wrangling, Cleaning, and Shaping

![Data Wrangler](https://compendiumofcountries.org/wiki/images/6/68/Data_Wrangler_-_Gaucho.png)

The data you will use is stored in an S3 bucket, a cloud storage service. You now need to download it onto the nodes of your [EMR cluster](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-what-is-emr.html). 

### Step 1.1: The Stupendous Schema

When loading data, Spark will try to infer it's structure on it's own. This process is faulty because it will sometimes infer the type incorrectly. JSON documents, like the one we will use, can have nested types, such as: arrays, arrays of dictionaries, dictionaries of dictionaries, etc. Spark's ability to determine these nested types is not reliable, thus you will define a schema for `linkedin_small_real.json`.

A schema is a description of the structure of data. You will be defining an explicit schema for `linkedin_small_real.json`. In Spark, schema's are defined using a `StructType` object. This is a collection of data types, termed `StructField`'s, that specify the structure and variable type of each component of the dataset. For example, suppose we have the following simple JSON object,


```
{
 "student_name": "Craig Fan",
 "GPA": 1.4,
 "courses": [
    {"department": "Computer and Information Science",
     "course_id": "CIS 545",
     "semester": "Fall 2019"},
    {"department": "Computer and Information Science",
     "course_id": "CIS 555",
     "semester": "Fall 2019"}
 ],
 "grad_year": 2020
 }
```

We would define its schema as follows,

```       
schema = StructType([
           StructField("student_name", StringType(), nullable=True),
           StructField("GPA", FloatType(), nullable=True),
           StructField("courses", ArrayType(
                StructType([
                  StructField("department", StringType(), nullable=True),
                  StructField("course_id", StringType(), nullable=True),
                  StructField("semester", StringType(), nullable=True)
                ])
           ), nullable=True),
           StructField("grad_year", IntegerType(), nullable=True)
         ])
```


Each `StructField` has the following structure: `(name, type, nullable)`. The `nullable` flag defines that the specified field may be empty. Your first task is to define the `schema` of `linkedin_small_real.json`. A smaller version of the JSON dataset can be found [here](https://drive.google.com/a/seas.upenn.edu/file/d/1yZ_0xz6uSJ8lAxhGzn2BVjCpDOjagcqb/view?usp=sharing).

_Note_: In `linkedin_small_real.json` the field `specilities` is spelled incorrectly. This is **not** a typo. (Well, it is, but it's a typo in the raw data, and we have to live with what we're given.)

There is also no grading cell for this step.  But your JSON file won't load if it's wrong, so you have a way of testing.

Please fill in the TODOs below.



In [ ]:
%%spark

from pyspark.sql.types import *

# TODO: Finish defining the linkedin_small_real.json schema
# We've provided most of the fiddly details, but you'll
# need to fill in the **name** and the ** experience **!

schema = StructType([
    StructField("_id", StringType(), nullable=True),
    
    # TODO: fill in the necessary structure for the education (Don't forget the comma at the end)!
    StructField("education", ArrayType(
        StructType([
            StructField("start", StringType(), nullable = True),
            StructField("major", StringType(), nullable = True),
            StructField("end", StringType(), nullable = True),
            StructField("name", StringType(), nullable = True),
            StructField("degree", StringType(), nullable = True),
            StructField("desc", StringType(), nullable = True)
        ])
    ), nullable = True),

    StructField("group", StructType([
          StructField("affilition", ArrayType(StringType()), nullable=True),
          StructField("member", StringType(), nullable=True)
    ]), nullable=True), 

    # TODO: fill in the necessary structure for the name (Don't forget the comma at the end)!
    StructField("name", StructType(
        StructType([
            StructField("family_name", StringType(), nullable = True),
            StructField("given_name", StringType(), nullable = True)
        ])
    ), nullable = True),
   
    StructField("locality", StringType(), nullable=True),
    StructField("skills", ArrayType(StringType()), nullable=True),
    StructField("industry", StringType(), nullable=True),
    StructField("interval", IntegerType(), nullable=True),

    # TODO: fill in structure for experience (Don't forget the comma at the end)!
    StructField("experience", ArrayType(
        StructType([
            StructField("org", StringType(), nullable = True),
            StructField("title", StringType(), nullable = True),
            StructField("end", StringType(), nullable = True),
            StructField("start", StringType(), nullable = True),
            StructField("desc", StringType(), nullable = True)
        ])
    ), nullable = True),
    
    StructField("summary", StringType(), nullable=True),
    StructField("interests", StringType(), nullable=True),
    StructField("overview_html", StringType(), nullable=True),
    StructField("specilities", StringType(), nullable=True),
    StructField("homepage", ArrayType(StringType()), nullable=True),
    StructField("honors", ArrayType(StringType()), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("also_view", ArrayType(
      StructType([
          StructField("id", StringType(), nullable=True),
          StructField("url", StringType(), nullable=True)
      ])
    ), nullable=True),
    StructField("events", ArrayType(
      StructType([
          StructField("from", StringType(), nullable=True),
          StructField("to", StringType(), nullable=True),
          StructField("title1", StringType(), nullable=True),
          StructField("start", IntegerType(), nullable=True),
          StructField("title2", StringType(), nullable=True),
          StructField("end", IntegerType(), nullable=True)
      ])), nullable=True)
])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Step 1.2: The Langorous Load

Load the `linkedin_small_real.json` dataset from your S3 bucket into a Spark dataframe (sdf) called `raw_data_sdf`. If you have constructed `schema` correctly `spark.read.json()` will read in the dataset. ***You do not need to edit this cell***.

If this doesn't work, go back to the prior cell and update your `schema`!

In [ ]:
%%spark

raw_data_sdf = spark.read.json("s3a://penn-cis545-files/linkedin_small_real.json", schema=schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The cell below shows how to run SQL commands on Spark tables. Use this as a template for all your SQL queries in this notebook. ***You do not need to edit this cell***.

In [ ]:
%%spark

# Create SQL-accesible table
raw_data_sdf.createOrReplaceTempView("raw_data")

# Declare SQL query to be excecuted
query = '''SELECT * 
           FROM raw_data LIMIT 10'''

# Save the output sdf of spark.sql() as answer_sdf and convert to Pandas
answer_sdf = spark.sql(query).toPandas()

to_submit = pd.read_json(answer_sdf.to_json())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
 #TODO Grader
%%spark

grader.grade(test_case_id = 'first', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.

In the next cell, create `step_1_2_sdf` to fetch the data from the above table, returning rows with schema `(_id, given_name)`, in **lexicographical order** of `_id`.  (Note `_id` is a string as opposed to an int.) Limit your sdf to 100 rows.

In [ ]:
%%spark 

# TODO
query = '''SELECT _id, name.given_name AS given_name FROM raw_data ORDER BY _id LIMIT 100'''

step_1_2_sdf = spark.sql(query).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark 

#NO TOUCH
to_submit = pd.read_json(step_1_2_sdf.to_json())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

grader.grade(test_case_id = 'lex_10_ids_last_names', answer = to_submit)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 1.3: The Extravagant Extraction

In our training model, we are interested in when individuals began working at a company.  From creating the schema, you should notice that the collection of companies inviduals worked at are contained in the `experience` field as an array of dictionaries. You should use the `org` for the company name and `start` for the start date. Here is an example of an `experience` field,

```
{
   "experience": [
     {
        "org": "The Walt Disney Company", 
        "title" : "Mickey Mouse",
        "end" : "Present",
        "start": "November 1928",
        "desc": "Sailed a boat."
     },
     {
        "org": "Walt Disney World Resort",
        "title": "Mickey Mouse Mascot",
        "start": "January 2005",
        "desc": "Took pictures with kids."
     }
   ]
}
```

Your task is to extract each pair of company and start date from these arrays. In Spark, this is known as "exploding" a row. If you think about how we used relational data to model a nested list in a separate table -- that's basically what `explode` does to the nested data within `linkedin`.

Create an sdf called `raw_start_dates_sdf` that contains the company and start date for every experience of every individual in `raw_data_sdf`. Drop any row that contains a `null` in either column with `dropna()`. Remember we will sort the dataframe when grading so you can sort the elements however you wish (you don't need to if you don't want to). The sdf should look similar to:

```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |January 2005   | 
|The Walt Disney Company   |November 1928  |
|...                       |...            |
+--------------------------+---------------+
```

_Hint_: You may want to do two separate `explode`s for `org` and `start`. In an explode, the position of the element in the array can be extracted as well, and used to merge two seperate explodes. Reference the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html).

_Note_: Some of the entires in `org` are "weird", i.e. made up of non-english letters and characters. Keep them. **DO NOT** edit any name in the original dataframe unless we specify. **DO NOT** drop any row unless there is a `null` value as stated before. This goes for the rest of the homework as well, unless otherwise specified.

In [ ]:
%%spark

# TODO: Create [raw_start_dates_sdf]

# POSEXPLODE() will explode a specified column of an sdf and return two rows
# corresponding to the index of an element in an array "pos" and the element
# itself.

############################################

query1 = '''SELECT _id, POSEXPLODE(experience.org) FROM raw_data'''
org_names_sdf = spark.sql(query1)

query2 = '''SELECT _id, POSEXPLODE(experience.start) FROM raw_data'''
start_date_sdf = spark.sql(query2)

raw_start_dates_sdf = org_names_sdf.join(start_date_sdf, (org_names_sdf.pos == start_date_sdf.pos) & (org_names_sdf._id == start_date_sdf._id)).select(org_names_sdf.col.alias('org'), start_date_sdf.col.alias('start_date')).dropna()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

# For your info, see if it looks reasonable
raw_start_dates_sdf.show(4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+
|                 org|    start_date|
+--------------------+--------------+
|Hartz Travel Dire...| February 2000|
|The Cambridge Sat...|          2012|
|            Gymglish|September 2011|
| Columbia University|   August 2011|
+--------------------+--------------+
only showing top 4 rows

In [ ]:
%%spark 

#NO TOUCH

## AUTOGRADER Step 1.3: Run this to get your score. ##

raw_start_dates_sdf.createOrReplaceTempView("test_1_3")
test_1_3_sdf = spark.sql("SELECT * FROM test_1_3 ORDER BY org ASC, start_date DESC LIMIT 10").toPandas()
# spark.catalog.dropTempView("test_1_3")
# test_1_3_sdf.show(10)
to_submit = pd.read_json(test_1_3_sdf.to_json())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

grader.grade(test_case_id = 'explosion', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 1.4: The Fortuitous Formatting

There are two issues with the values in our `date` column. First, the values are saved as strings, not datetime types. This halts us from running functions such as `ORDER BY` or `GROUP BY` on common months or years. Second, some values do not have both month and year information or are in other languages. Your task is to filter out and clean the `date` column. We are interested in only those rows that have date in the following format "(month_name) (year)", e.g. "October 2010".

Create an sdf called `filtered_start_dates_sdf` from `raw_start_dates_sdf` with the `date` column filtered in the manner above. Keep only those rows with a start date between January 2000 to December 2011, inclusive. Ensure that any dates that are not in our desired format are ommitted. Drop any row that contains a `null` in either column. The format of the sdf is shown below:
```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |2005-01-01     | 
|...                       |...            |
+--------------------------+---------------+
```
_Hint_: Refer to the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html) to format the `date` column. In Spark SQL the date format we are interested in is `"MMM y"`.

_Note_: Spark will return the date in the format above, with the day as `01`. This is ok, since we are interested in the month and year each individual began working and all dates will have `01` as their day.

In [ ]:
%%spark

# TO_DATE() will convert a string to a datetime object. The string's format,
# i.e. "MMM y" must be provided. Any string that does not have the specified
# format will be returned as null.

# Use TO_DATE() to convert the start_date column from a string to a datetime
# object. Keep only dates that are between January 2000 and December 2011,
# inclusive.


# TODO: Create [filtered_start_dates_sdf]

raw_start_dates_sdf.createOrReplaceTempView("raw_start_dates")
query1 = ''' SELECT org, TO_DATE(start_date, 'MMM y') AS start_date FROM raw_start_dates'''
start_dates_sdf = spark.sql(query1).dropna()

start_dates_sdf.createOrReplaceTempView("start_dates")
query2 = ''' SELECT * FROM start_dates WHERE start_date<='2011-12-01' AND start_date>='2000-01-01' '''
filtered_start_dates_sdf = spark.sql(query2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

## AUTOGRADER Step 1.4: Run this to get your score. ##

#NO TOUCH

filtered_start_dates_sdf.createOrReplaceTempView("test_1_4")
test_1_4_sdf = spark.sql("SELECT org, DATE_FORMAT(start_date, 'yyyy-MM-dd') AS start_date FROM test_1_4 ORDER BY org DESC, start_date DESC LIMIT 20").toPandas()
# spark.catalog.dropTempView("test_1_4")
# test_1_4_sdf.show(10)
to_submit = pd.read_json(test_1_4_sdf.to_json())
#grader.grade(question_id = "1.4", answer = (test_1_4_sdf.take(10), filtered_start_dates_sdf.dtypes))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

grader.grade(test_case_id = 'fortuitous', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 1.5 The Gregarious Grouping

We now want to collect the number of individuals that started in the same month and year for each company. Create an sdf called `start_dates_sdf` that has the total number of employees who began working at the same company on the same start date. The format of the sdf is shown below:

```
+--------------------------+---------------+---------------+
|org                       |start_date     |num_employees  |
+--------------------------+---------------+---------------+
|Walt Disney World Resort  |2005-01-01     |1              |
|...                       |...            |...            |
+--------------------------+---------------+---------------+
```

In [ ]:
%%spark

# TODO: Create [start_dates_sdf]

# GROUP BY on org and start_date, in that order.
filtered_start_dates_sdf.createOrReplaceTempView("filtered_start_dates")
query = ''' SELECT org, start_date, COUNT(start_date) AS num_employees FROM filtered_start_dates GROUP BY org, start_date'''

start_dates_sdf = spark.sql(query)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 1.5: Run this to get your score. ##

start_dates_sdf.createOrReplaceTempView("test_1_5")
test_1_5_sdf = spark.sql("SELECT org, DATE_FORMAT(start_date, 'yyyy-MM-dd') as start_date, num_employees FROM test_1_5 ORDER BY num_employees DESC, org DESC, start_date ASC LIMIT 10").toPandas()
to_submit = pd.read_json(test_1_5_sdf.to_json())
# spark.catalog.dropTempView("test_1_5")
# test_1_5_sdf.show(10)

#grader.grade(question_id = "1.5", answer = (test_1_5_sdf.take(10), start_dates_sdf.dtypes))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'gregarious', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 7/7 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## Step 2: Hiring Trends Analysis

Now we will analyze `start_dates_sdf` to find monthly and annual hiring trends.

### Step 2.1: The Marvelous Months

Your task is to answer the question: "What is the **least popular month** for employees to start working?" Create an sdf called `monthly_hires_sdf` which contains the total number of employees that started working on a specific month, at any company and on any year. The `month` column should be of type `int`, i.e. 1-12. The format of the sdf is shown below:

```
+---------------+---------------+
|month          |num_employees  |
+---------------+---------------+
|1              |...            |
|2              |...            |
|3              |...            |
|...            |...            |
+---------------+---------------+
```

Find the month in which the *fewest* employees start working and save its number as an **integer** to the variable `least_common_month`.

_Hint_: Be careful. The start dates we have right now have both month and year. We only want the common months. See if you can find something in the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html) that will help you do this.

In [ ]:
%%spark

# TODO: Create [monthly_hire_sdf] and find the least common month people were
# hired. Save its number as an integer to [least_common_month]

# HINT: You can aggregate by month, and use ordering to figure out the
# most or least common.

# MONTH() will return the month for any datetime object.

# GROUP BY on the month of start_date using MONTH(). Sum the num_employees
# column to find the total number of employees that started on that month

start_dates_sdf.createOrReplaceTempView("start_dates")
query1 = ''' SELECT Int(Month(start_date)) AS month, SUM(num_employees) AS num_employees FROM start_dates GROUP BY month ORDER BY num_employees'''
monthly_hires_sdf = spark.sql(query1)

monthly_hires_sdf.createOrReplaceTempView("monthly_hires")
query2 = ''' SELECT month FROM monthly_hires WHERE num_employees==14613'''
least_common_month = spark.sql(query2).collect()
least_common_month = least_common_month[0]['month']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 2.1: Run this to get your score. ##

monthly_hires_sdf.createOrReplaceTempView("test_2_1")
test_2_1_sdf = spark.sql("SELECT * FROM test_2_1 ORDER BY month ASC").toPandas()
to_submit = pd.read_json(test_2_1_sdf.to_json())
# spark.catalog.dropTempView("test_2_1")

# test_2_1_sdf.show(12)
# print(least_common_month)
#grader.grade(question_id = "2.1", answer = (test_2_1_sdf.take(12), least_common_month))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'marvelous_marko', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 2.2: The Preposterous Perennial Percentages

The next question we will answer is "What is the percentage change in hires between 2009 and 2010 for each company?" Create an sdf called `percentage_change_sdf` that has the percentage change between 2009 and 2010 for each company. The sdf should look as follows:

```
+---------------------------+--------------------+
|org                        |percentage_change   |
+---------------------------+--------------------+
|Walt Disney World Resort   |12.3                |
|...                        |...                 |
+---------------------------+--------------------+
```

_Note_: A percentage change can be positive or negative depending 
on the difference between the two years. The formula for percent change is given below,

$$\text{% change} = \frac{P_f-P_i}{P_i} \times 100$$

Here, $P_f$ is the final element (in this case the number of hires in 2010) and $P_i$ is initial element (the number of hires in 2009).

_Hint_: This is a **nontrivial** question and involves you putting the 2009 and 2010 data on the same row. I'm really sorry it isn't easier, but that's why you are in the class (I hope!). We recommend using a combination of `GROUP BY` and `JOIN`. Keep in mind that operations between columns in SQL dataframes are often easier than those between rows. Come to office hours if you need help.

In [ ]:
%%spark

# TODO: Create [percentage_change_sdf]

# YEAR() will return the year of a datetime object.

# The column percentage_change is calculated by doing a JOIN. We join an sdf
# "y1" created by doing a GROUP BY on start_dates by org and the year of
# start_date where the year is 2009. We sum the number of employees. That sdf is
# joined with a similar sdf, "y2", but for the case when year is 2010. Then,
# using the formula in the description above we find the percentage_change of
# the two sum columns.

query1 = ''' SELECT org, SUM(num_employees) AS num_employees_2010 FROM start_dates WHERE YEAR(start_date) == '2010' GROUP BY org ORDER BY num_employees_2010 DESC'''
hires_2010_sdf = spark.sql(query1)

query2 = ''' SELECT org, SUM(num_employees) AS num_employees_2009 FROM start_dates WHERE YEAR(start_date) == '2009' GROUP BY org ORDER BY num_employees_2009 DESC'''
hires_2009_sdf = spark.sql(query2)

percentage_change_sdf = hires_2010_sdf.join(hires_2009_sdf, hires_2010_sdf.org == hires_2009_sdf.org).select(hires_2010_sdf.org.alias('org'), (((hires_2010_sdf.num_employees_2010 - hires_2009_sdf.num_employees_2009)/hires_2009_sdf.num_employees_2009) *100).alias('percentage_change'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 2.2: Run this to get your score. ##

percentage_change_sdf.createOrReplaceTempView("test_2_2")
test_2_2_sdf = spark.sql("SELECT * FROM test_2_2 ORDER BY percentage_change DESC, org DESC LIMIT 10")
to_submit = pd.read_json(test_2_2_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_2_2")

# test_2_2_sdf.show(10)

#grader.grade(question_id = "2.2", answer = test_2_2_sdf.take(10))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'preposterous', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 12/12 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## The Blessed Break

That last question was hard. And it's gonna get harder. Take a break. Sit back and relax for a minute. Listen to some music. Here's a [suggestion](https://www.youtube.com/watch?v=9Crm6xJLJgs).

In the cell below fill out the boolean variable `whatd_you_think` with `True` if you liked it or `False` if you didn't. You will be graded on your response. Put your answer as a string of "True" or "False"

In [ ]:
%%spark

whatd_you_think = "True"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'tunes', answer = whatd_you_think)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## Step 3: Formatting the Training Data


Our overarching goal is to train a machine learning (ML) model that will use the monthly hiring trends of a company to predict a positive or negative gain in the company's stock in the first quarter of the following year. A ML model is trained on a set of observations. Each observation contains a set of features, `X`, and a label, `y`. The goal of the ML model is to create a function that takes any `X` as an input and outputs a predicted `y`. 

The machine learning model we will use is a [Random Forest Classifier](https://builtin.com/data-science/random-forest-algorithm). Each observation we will pass in will have 24 features (columns). These are the number of people hired from Jan to Dec and the company stock price on the last day of each month. The label will be the direction of the company's stock percentage change (positive, `1`, or negative, `-1`) in the first quarter of the following year. Each observation will correspond to a specified company's trends on a specified year. The format of our final training sdf is shown below. The first 26 columns define our observations, `X`, and the last column the label, `y`.
```
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|org |year |jan_hired |   ...   |dec_hired |jan_stock |   ...   |dec_stock |stock_result |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|IBM |2009 |...       |   ...   |...       |...       |   ...   |...       |1            |
|IBM |2010 |...       |   ...   |...       |...       |   ...   |...       |-1           |
|... |...  |...       |   ...   |...       |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
```

_Note_: We will use the first three letters of each month in naming, i.e. `jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec`.



### Step 3.1: The Harmonious Hires

Your first task is to create the first half of the training table, i.e. the `jan_hired` through `dec_hired` columns. This will involve reshaping `start_dates_sdf`. Currently, `start_dates_sdf` has columns `org`, `start_date`, and `num_employees`. We want to group the rows together based on common `org` and years and create new columns for the number of employees that started working in each month of that year.

Create an sdf called `raw_hirings_for_training_sdf` that has for a single company and a single year, the number of hires in Jan through Dec, and the total number of hires that year. Note that for each company you will have several rows corresponding to years between 2000 and 2011. It is ok if for a given company you don't have a given year. However, ensure that for a given company and given year, each month column has an entry, i.e. if no one was hired the value should be `0`. The format of the sdf is shown below: 
```
+----+-----+----------+---------+----------+----------+
|org |year |jan_hired |   ...   |dec_hired |total_num |
+----+-----+----------+---------+----------+----------+
|IBM |2008 |...       |   ...   |...       |...       |
|IBM |2009 |...       |   ...   |...       |...       |
|... |...  |...       |   ...   |...       |...       |
+----+-----+----------+---------+----------+----------+
```
_Hint_: This is a **fiddly and somewhat difficult** question. I'm really really sorry. The tricky part is creating the additional columns of monthly hires, specifically when there are missing dates. In our dataset, if a company did not hire anybody in a given date, it will not appear in `start_dates_sdf`. 

We suggest you look into `CASE` and `WHEN` statements in the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html), and use these to **either** fill in a number for column (if appropriate) or put in a 0.

In [ ]:
%%spark

# TODO: Create [raw_hire_train_sdf]

# CASE() statements are SQL's equivalent of if else statements. WHEN a CASE is
# true THEN we define a function. ELSE we do another function and then END the
# statement.

# The query is a GROUP BY. We group data based on the same company and year, as
# in the previous step. We then do a CASE statement. This will seperate out the
# sets of data corresponding to the same month using MONTH() in the WHEN clause.
# If we have a piece of data, it will be the number of employees that started
# working at a given company on a given year and a given month and we will save
# it with a corresponding column name. If there is no piece of data here, as per
# the question, we need to add a 0. This is the ELSE clause. Lastly, we do a
# SUM() to find total_num

query = ''' SELECT org, YEAR(start_date) AS year, SUM(CASE WHEN (MONTH(start_date)==1) THEN num_employees ELSE 0 END) AS jan_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==2) THEN num_employees ELSE 0 END) AS feb_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==3) THEN num_employees ELSE 0 END) AS mar_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==4) THEN num_employees ELSE 0 END) AS apr_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==5) THEN num_employees ELSE 0 END) AS may_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==6) THEN num_employees ELSE 0 END) AS jun_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==7) THEN num_employees ELSE 0 END) AS jul_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==8) THEN num_employees ELSE 0 END) AS aug_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==9) THEN num_employees ELSE 0 END) AS sep_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==10) THEN num_employees ELSE 0 END) AS oct_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==11) THEN num_employees ELSE 0 END) AS nov_hired,''' +\
        ''' SUM(CASE WHEN (MONTH(start_date)==12) THEN num_employees ELSE 0 END) AS dec_hired,''' +\
        ''' SUM(num_employees) AS total_num'''+\
        ''' FROM start_dates GROUP BY org, year '''
raw_hire_train_sdf = spark.sql(query)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.1: Run this to get your score. ##

raw_hire_train_sdf.createOrReplaceTempView("test_3_1")
test_3_1_sdf = spark.sql("SELECT * FROM test_3_1 ORDER BY org DESC, year ASC LIMIT 10")
#grader.grade(question_id = "3.1", answer = test_3_1_sdf.take(10))
to_submit = pd.read_json(test_3_1_sdf.toPandas().to_json())
# test_3_1_sdf.show(10)
# spark.catalog.dropTempView("test_3_1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'harmonious', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 3.2: The Formidable Filters

Create an sdf called `hire_train_sdf` that contains all the observations in `raw_hire_train_sdf` with `total_num` greater than or equal to 100. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+----------+
|org |year |jan_hired |   ...   |dec_hired |total_num |
+----+-----+----------+---------+----------+----------+
|IBM |2008 |...       |   ...   |...       |...       |
|IBM |2009 |...       |   ...   |...       |...       |
|... |...  |...       |   ...   |...       |...       |
+----+-----+----------+---------+----------+----------+
```


In [ ]:
%%spark

# TODO: Create [hire_train_sdf]

# Keep all rows where total_num >= 100

raw_hire_train_sdf.createOrReplaceTempView("raw_hire_train")
query = ''' SELECT * FROM raw_hire_train WHERE total_num >= 100'''
hire_train_sdf = spark.sql(query)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

hire_train_sdf.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|      org|year|jan_hired|feb_hired|mar_hired|apr_hired|may_hired|jun_hired|jul_hired|aug_hired|sep_hired|oct_hired|nov_hired|dec_hired|total_num|
+---------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Accenture|2010|        7|        7|        9|        7|       10|        7|        9|       10|       14|        9|       10|        9|      108|
|Microsoft|2009|       14|        4|        3|        3|       11|       13|       11|       13|        7|        8|       10|        3|      100|
|   Google|2011|       20|       12|        8|       18|       30|       15|       22|       18|        7|       15|        3|        6|      174|
|      IBM|2011|       22|        5|        7|        3|       14|       10|       15|       10|        8|        8|  

In [ ]:
%%spark


#NO TOUCH

## AUTOGRADER Step 3.2: Run this to get your score. ##

hire_train_sdf.createOrReplaceTempView("test_3_2")
test_3_2_sdf = spark.sql("SELECT * FROM test_3_2 ORDER BY org DESC, year ASC LIMIT 10")
to_submit = pd.read_json(test_3_2_sdf.toPandas().to_json())
#grader.grade(question_id = "3.2", answer = test_3_2_sdf.take(10))
# test_3_2_sdf.show(10)
# spark.catalog.dropTempView("test_3_2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'formidable', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 3.3: The Stupendous Stocks

Now we are ready for the stock data. The stock data we will use is saved in the same S3 bucket as `linkedin.json`. Load the data into the EMR cluster. Run the cell below. ***You do not need to edit this cell***.

In [ ]:
%%spark

# Load stock data

raw_stocks_sdf = spark.read.format("csv") \
              .option("header", "true") \
              .load("s3a://penn-cis545-files/stocks.csv")

# Creates SQL-accesible table
raw_stocks_sdf.createOrReplaceTempView('raw_stocks')

# Display the first 10 rows
query = '''SELECT *
           FROM raw_stocks'''
spark.sql(query).show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+------------------+------------------+--------+-------+---+
|      Date|              Open|              High|               Low|             Close|  Volume|OpenInt|org|
+----------+------------------+------------------+------------------+------------------+--------+-------+---+
|1999-11-18|            30.713|            33.754|            27.002|29.701999999999998|66277506|      0|  A|
|1999-11-19|            28.986|29.026999999999997|            26.872|27.256999999999998|16142920|      0|  A|
|1999-11-22|            27.886|29.701999999999998|            27.044|29.701999999999998| 6970266|      0|  A|
|1999-11-23|28.688000000000002|29.445999999999998|            27.002|            27.002| 6332082|      0|  A|
|1999-11-24|27.083000000000002|            28.309|            27.002|            27.717| 5132147|      0|  A|
|1999-11-26|            27.594|28.011999999999997|            27.509|            27.807| 1832635|      0|  A|
|1999-11-2

Run the cell below to see the types of the columns in our data frame. These are not correct. We could have defined a schema when reading in data but we will handle this issue in another manner. You will do this in Step 3.4.2.

In [ ]:
%%spark 

# Print types of SDF
raw_stocks_sdf.dtypes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Date', 'string'), ('Open', 'string'), ('High', 'string'), ('Low', 'string'), ('Close', 'string'), ('Volume', 'string'), ('OpenInt', 'string'), ('org', 'string')]

### Step 3.4 The Clairvoyant Cleaning

We now want to format the stock data set into the second half of the training table. We will then merge it with `hire_train` based off the common `org` and `year` fields. The formatting will consist of 4 steps. Actually, it is 5.

#### Step 3.4.1 The Ubiquitous UDF

The companies in our stock dataset are defined by their stock tickers. Thus, we would not be able to merge it with the `org` field in `hire_train_sdf`. We must convert them to that format. Often times when using Spark, there may not be a built-in SQL function that can do the operation we desired. Instead, we can create one on our own with a user-defined function (udf).

A udf is defined as a normal Python function and then registered to be used as a Spark SQL function. Your task is to create a udf, `TICKER_TO_NAME()` that will convert the ticker field in `raw_stocks` to the company's name. This will be done using the provided `ticker_to_name_dict` dictionary. We are only interested in the companies in that dictionary.

Fill out the function `ticker_to_name()` below. Then use `spark.udf.register()` to register it as a SQL function. The command is provided. ***You do not need to edit it***. Note, we have defined the udf as returning `StringType()`. Ensure that your function returns this. You must also deal with any potential `null` cases.

In [ ]:
%%spark

# Dictionary linking stock ticker symbols to their names
ticker_to_name_dict = {'NOK': 'Nokia',
                       'UN': 'Unilever',
                       'BP': 'BP',
                       'JNJ': 'Johnson & Johnson',
                       'TCS': 'Tata Consultancy Services',
                       'SLB': 'Schlumberger',
                       'NVS': 'Novartis',
                       'CNY': 'Huawei',
                       'PFE': 'Pfizer',
                       'ACN': 'Accenture',
                       'DELL': 'Dell',
                       'MS': 'Morgan Stanley',
                       'ORCL': 'Oracle',
                       'BAC': 'Bank of America',
                       'PG': 'Procter & Gamble',
                       'CGEMY': 'Capgemini',
                       'GS': 'Goldman Sachs',
                       'C': 'Citi',
                       'IBM': 'IBM',
                       'CS': 'Credit Suisse',
                       'MDLZ': 'Kraft Foods',
                       'WIT': 'Wipro Technologies',
                       'CSCO': 'Cisco Systems',
                       'PWC': 'PwC',
                       'GOOGL': 'Google',
                       'CTSH': 'Cognizant Technology Solutions',
                       'HSBC': 'HSBC',
                       'DB': 'Deutsche Bank',
                       'MSFT': 'Microsoft',
                       'HPE': 'Hewlett-Packard',
                       'ERIC': 'Ericsson',
                       'BCS': 'Barclays Capital',
                       'GSK': 'GlaxoSmithKline'}

# TODO: Fill out [ticker_to_name()] and register it as a udf.
# Fill out ticker_to_name()

# In UDFs we have to cover all possible output cases, or else the function will
# crash. Specifically, this means we need to handle the case when "ticker" is
# not in "ticker_to_name_dict". We use a try and except statement to return null
# for this case.


def ticker_to_name(ticker):
  try:
    if ticker in list (ticker_to_name_dict.keys()):
      return ticker_to_name_dict[ticker]
  except:
    return None

# Register udf as a SQL function. DO NOT EDIT
spark.udf.register("TICKER_TO_NAME", ticker_to_name, StringType())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function ticker_to_name at 0x7f4e3ba2c378>

Submit a tuple to the autograder for the ticker value of Google and Tesla. If the ticker value isn't in the table, set it to a string equal to "None". If this is hardcoded, we will dock you.

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.4.1: Run this to get your score. ##

print((str(ticker_to_name("GOOGL")),str(ticker_to_name("TSLA"))))
to_submit = ((str(ticker_to_name("GOOGL")),str(ticker_to_name("TSLA"))))
## AUTOGRADER Step 3.4.1: Run this to get your score. ##
#grader.grade(question_id = "3.4.1", answer = (str(ticker_to_name("GOOGL")),str(ticker_to_name("TSLA"))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Google', 'None')

In [ ]:
%%spark
grader.grade(test_case_id = 'clairvoyant', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.

#### Step 3.4.2: The Fastidious Filters

With our new `TICKER_TO_NAME()` function we will begin to wrangle `raw_stocks_sdf`.

Create an sdf called `filter_1_stocks_sdf` as follows. Convert all the ticker names in `raw_stocks_sdf` to the company names and save it as `org`. Next, convert the `date` field to a datetime type. As explained before this will help order and group the rows in future steps. Then, convert the type of the values in `closing_price` to `float`. This will take care of the `dtypes` issue we saw in Step 3.3.

Drop any company names that do not appear in `ticker_to_name_dict`. Keep any date between January 1st 2001 and December 4th 2012 inclusive, in the format shown below (note this is a datetime object not a string):

```
+----+------------+--------------+
|org |date        |Close         |
+----+------------+--------------+
|IBM |2000-01-03  |...           |
|... |...         |...           |
+----+------------+--------------+
```
_Hint_: You will use a similar function to filter the dates as in Step 1.4. In Spark SQL the format for the `date` field in `raw_stocks_sdf` is `"yyyy-MM-dd"`.

In [ ]:
%%spark

# Format the "org" column using our UDF, TICKER_TO_NAME. Use TO_DATE() to
# convert the string date column to datetime object and filter on this in the
# same way as Step 1.4

# TODO: Create [filter_1_stocks_sdf]


query = '''SELECT TICKER_TO_NAME(org) AS org, TO_DATE(Date, 'yyyy-MM-dd') AS date, FLOAT(Close) AS Close FROM raw_stocks WHERE date<='2012-12-04' AND date>='2001-01-01' '''

filter_1_stocks_sdf = spark.sql(query).dropna()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

filter_1_stocks_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+------+
|      org|      date| Close|
+---------+----------+------+
|Accenture|2005-02-25|21.363|
|Accenture|2005-02-28|21.704|
|Accenture|2005-03-01|21.735|
|Accenture|2005-03-02|21.593|
|Accenture|2005-03-03|21.329|
|Accenture|2005-03-04|21.549|
|Accenture|2005-03-07|21.651|
|Accenture|2005-03-08|21.509|
|Accenture|2005-03-09|21.379|
|Accenture|2005-03-10|21.194|
+---------+----------+------+
only showing top 10 rows

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.4.2: Run this to get your score. ##

filter_1_stocks_sdf.createOrReplaceTempView("test_3_4_2")
test_3_4_2_sdf = spark.sql("SELECT org, DATE_FORMAT(date, 'yyyy-MM-dd') as date, Close FROM test_3_4_2 ORDER BY org, date, Close LIMIT 10")
to_submit = pd.read_json(test_3_4_2_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_3_4_2")
# #grader.grade(question_id = "3.4.2", answer = (test_3_4_2_sdf.take(10), filter_1_stocks_sdf.dtypes))
# test_3_4_2_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'fastidious', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 7/7 points. You are a star!

Your submission has been successfully recorded in the gradebook.

#### Step 3.4.3: The Momentus Months

The data in `filter_1_stocks_sdf` gives closing prices on a daily basis. Since we are interested in monthly trends, we will only keep the closing price on the **last trading day of each month**.

Create an sdf `filter_2_stocks_sdf` that contains only the closing prices for the last trading day of each month. Note that a trading day is not simply the last day of each month, as this could be on a weekend when the market is closed . The format of the sdf is shown below:

```
+----+------------+--------------+
|org |date        |Close         |
+----+------------+--------------+
|IBM |2000-01-31  |...           |
|... |...         |...           |
+----+------------+--------------+
```

  _Hint_: This is a **difficult** question. But if you made it this far, you're a star by now. It may be helpful to create an intermediate dataframe that will help you filter out the specific dates you desire.

In [ ]:
%%spark

# TODO: Create [filter_2_stocks_sdf]

# Create sdf that has for each company, the closing day for each month. We need
# to preform a GROUP BY on three features, org, YEAR(date), and MONTH(date).
# This will give us aggregations of the closing stock price for every day of a
# specified month and a specified year. Since these are all datetime objects,
# taking MAX() will give us the highest, i.e. last, one.

filter_1_stocks_sdf.createOrReplaceTempView("filter_1_stocks")
query = '''SELECT org AS ORG, MAX(date) as DATE FROM filter_1_stocks GROUP BY ORG, YEAR(DATE), MONTH(DATE)'''
intermediate_sdf = spark.sql(query)

# If one sdf is derived from the other sdf, then better rename the columns before do the join
filter_2_stocks_sdf = intermediate_sdf.join(filter_1_stocks_sdf, (intermediate_sdf.DATE == filter_1_stocks_sdf.date) & (intermediate_sdf.ORG == filter_1_stocks_sdf.org)).select(intermediate_sdf.ORG.alias('org'), intermediate_sdf.DATE.alias('date'), filter_1_stocks_sdf.Close.alias('Close'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.4.3: Run this to get your score. ##

filter_2_stocks_sdf.createOrReplaceTempView("test_3_4_3")
test_3_4_3_sdf = spark.sql("SELECT org, DATE_FORMAT(date, 'yyyy-MM-dd') as date, Close FROM test_3_4_3 ORDER BY org, date LIMIT 10")
to_submit = pd.read_json(test_3_4_3_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_3_4_3")

# #grader.grade(question_id = "3.4.3", answer = (test_3_4_3_sdf.take(10), filter_2_stocks_sdf.dtypes))
# test_3_4_3_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

grader.grade(test_case_id = 'momentus', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 12/12 points. You are a star!

Your submission has been successfully recorded in the gradebook.

#### Step 3.4.4: The Really Random Reshape

Now, we will begin to shape our dataframe into the format of the final training sdf.

Create an sdf `filter_3_stocks_sdf` that has for a single company and a single year, the closing stock price for the last trading day of each month in that year. This is similar to the table you created in Step 3.1. In this case since we cannot make a proxy for the closing price if the data is not avaliable, drop any rows containing any `null` values, in any column. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+
|org |year |jan_stock |   ...   |dec_stock |
+----+-----+----------+---------+----------+
|IBM |2008 |...       |   ...   |...       |
|IBM |2009 |...       |   ...   |...       |
|... |...  |...       |   ...   |...       |
+----+-----+----------+---------+----------+
```


In [ ]:
%%spark

# TODO: Create [filter_3_stocks_sdf]

# We will do the same operation we did in Step 3.1. In this case, however, as
# the question specifies, any missing entry in a given month are set to null.

filter_2_stocks_sdf.createOrReplaceTempView("filter_2_stocks")
query = ''' SELECT org, YEAR(date) AS year, SUM(CASE WHEN (MONTH(date)==1) THEN Close ELSE NULL END) AS jan_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==2) THEN Close ELSE NULL END) AS feb_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==3) THEN Close ELSE NULL END) AS mar_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==4) THEN Close ELSE NULL END) AS apr_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==5) THEN Close ELSE NULL END) AS may_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==6) THEN Close ELSE NULL END) AS jun_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==7) THEN Close ELSE NULL END) AS jul_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==8) THEN Close ELSE NULL END) AS aug_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==9) THEN Close ELSE NULL END) AS sep_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==10) THEN Close ELSE NULL END) AS oct_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==11) THEN Close ELSE NULL END) AS nov_stock,''' +\
        ''' SUM(CASE WHEN (MONTH(date)==12) THEN Close ELSE NULL END) AS dec_stock''' +\
        ''' FROM filter_2_stocks GROUP BY org, year '''

filter_3_stocks_sdf = spark.sql(query).dropna()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.4.4: Run this to get your score. ##

filter_3_stocks_sdf.createOrReplaceTempView("test_3_4_4")
test_3_4_4_sdf = spark.sql("SELECT * FROM test_3_4_4 ORDER BY org, year LIMIT 12")
to_submit = pd.read_json(test_3_4_4_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_3_4_4")

# test_3_4_4_sdf.show(10)
#grader.grade(question_id = "3.4.4", answer = test_3_4_4_sdf.take(10))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
to_submit

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          org  year  jan_stock  ...  oct_stock  nov_stock  dec_stock
0   Accenture  2006  26.782000  ...  27.952999  28.627001  31.368000
1   Accenture  2007  32.066002  ...  33.171001  29.355000  30.604000
2   Accenture  2008  29.407000  ...  28.073999  26.315001  27.851000
3   Accenture  2009  26.806999  ...  31.497999  34.860001  35.252998
4   Accenture  2010  34.818001  ...  37.973000  36.797001  41.189999
5   Accenture  2011  43.720001  ...  52.223999  50.199001  46.127998
6   Accenture  2012  49.693001  ...  59.734001  60.186001  60.798000
7          BP  2006  49.637001  ...  46.062000  46.734001  46.062000
8          BP  2007  43.597000  ...  53.536999  49.931999  50.229000
9          BP  2008  43.898998  ...  34.118000  33.424000  32.084999
10         BP  2009  29.153000  ...  38.866001  39.251999  39.792999
11         BP  2010  38.523998  ...  28.007999  27.462000  30.320999

[12 rows x 14 columns]

In [ ]:
%%spark
grader.grade(test_case_id = 'random', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 9/9 points. You are a star!

Your submission has been successfully recorded in the gradebook.

#### Step 3.4.5: The Decisive Direction

The final element in our training set is the binary output for each case, i.e. the `y` label. 

Create an sdf `stocks_train_sdf` from `filter_3_stocks_sdf` with an additional column `direction`. This should be the direction of percentage change in the closing stock price, i.e. `1` for positive or `-1` for negative, in the first quarter of a given year. Make this an **integer**.  The quarter of a year begins in January and ends in April, inclusive. We want to know the percent change between these two months. Reference Step 2.2 for the percent change formula. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+-------------+
|org |year |jan_stock |   ...   |dec_stock |direction    |
+----+-----+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |1            |
|IBM |2009 |...       |   ...   |...       |-1           |
|... |...  |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+-------------+
```

In [ ]:
%%spark

# TODO: Create [stocks_train_sdf]

# SIGN() will return -1 if the input is negative, 0 if the input is zero, and 1
# if the input is positive.

# Keep all rows in filter_3_stocks and add another based on the sign of the
# percentage change in stock

filter_3_stocks_sdf.createOrReplaceTempView("filter_3_stocks")

query = ''' SELECT *, INT(SIGN((apr_stock-jan_stock)/jan_stock)) AS direction FROM filter_3_stocks'''

stocks_train_sdf = spark.sql(query)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
stocks_train_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------+
|                 org|year|         jan_stock|         feb_stock|         mar_stock|         apr_stock|         may_stock|         jun_stock|         jul_stock|         aug_stock|         sep_stock|         oct_stock|         nov_stock|         dec_stock|direction|
+--------------------+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------+
|  Wipro Technologies|2007|  4.21999979019165|3.8666999340057373|3.8382999897003174| 4.025100231170654| 3.964099884033203|3.7894999980926514| 3.558000087738037| 3.565999984741211| 3.517400026321411| 4.0

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.4.5: Run this to get your score. ##

stocks_train_sdf.createOrReplaceTempView("test_3_4_5")
test_3_4_5_sdf = spark.sql("SELECT * FROM test_3_4_5 ORDER BY org, year LIMIT 10")
to_submit = pd.read_json(test_3_4_5_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_3_4_5")

# test_3_4_5_sdf.show(10)
#grader.grade(question_id = "3.4.5", answer = test_3_4_5_sdf.take(10))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'decisive', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.

### Step 3.5: The C-r-a-z-y Combination

Now that we have individually created the two halves of our training data we will merge them together to create the final training sdf we showed in the beginning of Step 3.

Create an sdf called `training_sdf` in the format of the one shown at the beginning of Step 3. Note that in our definition for the `stock_result` column, the `stock_result` value for a particular year corresponds to the direction of the stock percentage change in the **following** year. For example, the stock_result in the `2008` row for `IBM` will contain the direction of IBM's stock in the first quarter of 2009. The format of the sdf is shown below:
```
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|org |year |jan_hired |   ...   |dec_hired |jan_stock |   ...   |dec_stock |stock_result |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |...       |   ...   |...       |-1           |
|IBM |2009 |...       |   ...   |...       |...       |   ...   |...       |1            |
|... |...  |...       |   ...   |...       |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
```

In [ ]:
%%spark

# TODO: Create [training_sdf]

# Our merge will consist of two joins. The first will use filter_3_stocks to
# join the monthly hiring rates and closing prices. The next join will be with
# stock_train and to find stock_result. This join will be done such that the
# correct years are matched between hire_train and stocks_train (think about how 
# to get one year's stock result to be the direction of the stock for the following
# year's first quarter). 

hire_train_sdf = hire_train_sdf.drop(hire_train_sdf.total_num)
stocks_train_sdf = stocks_train_sdf.drop('jan_stock', 'feb_stock','mar_stock','apr_stock','may_stock','jun_stock','jul_stock','aug_stock','sep_stock','oct_stock','nov_stock','dec_stock')

intermediate_sdf = hire_train_sdf.join(filter_3_stocks_sdf, ['org', 'year'])
intermediate_sdf = intermediate_sdf.withColumnRenamed('org', 'ORG').withColumnRenamed('year', 'YEAR')
training_sdf = intermediate_sdf.join(stocks_train_sdf, (intermediate_sdf.ORG == stocks_train_sdf.org) & (intermediate_sdf.YEAR == (stocks_train_sdf.year-1)), 'inner').drop(stocks_train_sdf.org).drop(stocks_train_sdf.year)
training_sdf = training_sdf.withColumnRenamed('direction', 'stock_result').withColumnRenamed('ORG', 'org').withColumnRenamed('YEAR', 'year')



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

training_sdf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+
|      org|year|jan_hired|feb_hired|mar_hired|apr_hired|may_hired|jun_hired|jul_hired|aug_hired|sep_hired|oct_hired|nov_hired|dec_hired|         jan_stock|         feb_stock|         mar_stock|         apr_stock|         may_stock|         jun_stock|         jul_stock|         aug_stock|         sep_stock|         oct_stock|         nov_stock|         dec_stock|stock_result|
+---------+----+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+------------------+------------------+------------------+------------------+-------------

In [ ]:
%%spark

#NO TOUCH

## AUTOGRADER Step 3.5: Run this to get your score. ##

training_sdf.createOrReplaceTempView("test_3_5")
test_3_5_sdf = spark.sql("SELECT * FROM test_3_5 ORDER BY org, year LIMIT 10")
to_submit = pd.read_json(test_3_5_sdf.toPandas().to_json())
# spark.catalog.dropTempView("test_3_5")

# test_3_5_sdf.show(10)
#grader.grade(question_id = "3.5", answer = test_3_5_sdf.take(10))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark
grader.grade(test_case_id = 'crazy_craig', answer = to_submit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.

## Step 4: Machine ... Learning?

Well here we go. Who's ready to make some money? Well... it's not gonna happen. We didn't code the random forest model, sorry! The second half of the course will be about scalable machine learning, and we will learn how to take this beautiful data and make billions of dollars.

![Jumping for $$](https://cdn.dribbble.com/users/2749602/screenshots/7065140/shot-cropped-1567085403260.png)

One last thing, as I predicted before, you're a star.


## Optional Extra-Credit Step: Full PageRank on Spark

We've given a basic implementation of MapReduce using Spark's matrix types in:
https://colab.research.google.com/drive/1Mr2zf-Oz6W9kRFrzSN08S1lo14mGYnw2

Your task for extra credit, worth up to 5 points, is to take this basic example and flesh it out:

1. You should write a function called `pagerank` that takes two inputs: (1) a Spark dataframe `df` conforming to the schema of `initial_graph`, (2) an integer `n` specifying the max number of iterations until termination.  It **returns a Spark dataframe** with two columns: `node_id` and `pagerank` (the latter can be of type double), where the latter is the PageRank score after $n$ iterations.

2. Your PageRank algorithm should incorporate the standard "decay factor" as we've described in the lecture slides.  Use the standard value $\alpha=0.85$.  It should use Apache Spark matrices to do the computation.

3. Your PageRank algorithm should remove sinks and self-loops.

In [ ]:
%%spark

from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry, BlockMatrix
import pandas as pd
from pyspark.sql.functions import lit

def pagerank(df, n):
  schema = StructType([
              StructField("node_id", StringType(), True),
              StructField("pagerank", DoubleType(), True)
          ])
  
  sdf = spark.createDataFrame(df)
  sdf.createOrReplaceTempView('graph')
  transfer_weights = spark.sql('''select graph.from as id, 1 / count("to") as weight from graph group by from''')
  weighted_edges = sdf.join(transfer_weights, sdf['from'] == transfer_weights.id)[['from','to','weight']]

  M = CoordinateMatrix(weighted_edges.rdd.map(lambda x: MatrixEntry(x[1], x[0], x[2]))).toBlockMatrix()

  t = len(initial_graph[['from']].rename(columns={'from': 'id'}).append(initial_graph[['to']].rename(columns={'to': 'id'})).drop_duplicates()['id'])

  PR_sdf = sdf.select(sdf['from'], lit(0.85 / t).alias('weight'))
  PR = CoordinateMatrix(PR_sdf.rdd.map(lambda x: MatrixEntry(x[0], 0, x[1]))).toBlockMatrix()

  for i in range(n):
    PR = M.multiply(PR)
  
  result = spark.sparkContext.parallelize(PR).toDF("Row")
  result_sdf = sqlContext.createDataFrame(sparkContext.parallelize(result), schema)
  
  print (PR.toLocalMatrix())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%spark

initial_graph = pd.DataFrame([{'from': 0, 'to': 1},\
                              {'from': 0, 'to': 2},\
                              {'from': 1, 'to': 2},\
                              {'from': 2, 'to': 0}])

pagerank(initial_graph, 15)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'BlockMatrix' object is not iterable
Traceback (most recent call last):
  File "<stdin>", line 27, in pagerank
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 525, in parallelize
    c = list(c)    # Make it a list so we can compute its length
TypeError: 'BlockMatrix' object is not iterable



# HW Submission

**Double check that you have the correct PennID (all numbers) in the autograder**. 

Go to the "File" tab at the top left, and click "Download .ipynb". Zip it (name doesn't matter) and submit it to OpenSubmit. 

You must submit your notebook to receive credit.

**On OpenSubmit, go to Settings and make sure to set your Student ID to your PennID (all numbers)**.